In [1]:
# !pip install -qq scrapy
# !pip install langchain
# !pip install -qU langchain-community faiss-cpu
# !pip install -qU langchain-openai
# !pip install --upgrade --quiet  rank_bm25
# !pip install langchain_experimental
# !pip install -U FlagEmbedding
# !pip install --upgrade --quiet  langchain-google-genai
# !pip install pyngrok
# !pip install flask_cors
# !pip install -U pinecone

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

ngrok_token = os.getenv('NGROK_TOKEN')
gemini_token = os.getenv('GEMINI_TOKEN')
pinecone_api = os.getenv('PINECONE_TOKEN')

In [2]:
import json

# Load the `output.json` file
with open('data.json', 'r',encoding="utf8") as f:
    data = json.load(f)

In [3]:
from uuid import uuid4
from bs4 import BeautifulSoup
from langchain_core.documents import Document

#remove excess characters and duplicate text
# import re
# for item in data:
#   item['content'] = re.sub(r'[^\S ]+', ' ', item['content'])


introduction = "Giới thiệu về cửa hàng: HerbaZen - Thương hiệu trà thảo mộc hiện đại, hướng đến lối sống xanh và quan tâm đến sức khỏe."
warranty = " HerbaZen cam kết mang đến những sản phẩm trà thảo mộc nguyên chất, được tuyển chọn kỹ lưỡng từ các vùng nguyên liệu sạch, không chất bảo quản, an toàn và tốt cho sức khỏe."
note = """Lưu ý khi sử dụng các dòng trà HerbaZen: 
-Nên uống sau khi ăn sáng và trước khi ngủ 30 phút để đạt hiệu quả tốt nhất.
-Không dùng khi đói hoặc thay thế hoàn toàn nước uống hàng ngày.
-Phụ nữ mang thai và người mắc bệnh mãn tính nên tham khảo ý kiến bác sĩ trước khi sử dụng.
-Đối với trà cam quế đường phèn: Không nên uống quá nhiều vào buổi tối vì quế có thể gây kích thích nhẹ
-Đối với trà an mộng: Không nên uống khi lái xe, cần tỉnh táo, Không nên dùng cùng lúc với thuốc an thần (có thể gây buồn ngủ quá mức)
-Đối với trà hoa hồng đường nâu-Trà dịu hồng: Không dùng cho người đang bị cảm nóng, sốt và Người có cơ địa nóng trong nên hạn chế đường nâu
-Đối với trà dưỡng nhan: Không nên uống khi đang có dấu hiệu nóng trong (nổi mụn, nhiệt miệng), Người bị tiểu đường nên hạn chế hoặc bỏ các thành phần chứa đường
"""
categories = "Các loại sản phẩm/Trà trong cửa hàng đang bán: Hoa hồng đường phèn - Hồng Tâm An, Hoa cúc táo đỏ kỳ tử - Cúc Ngọc Dưỡng Huyết,Trà an mộng ngủ ngon, Hoa hồng đường nâu - Trà dịu hồng"
list_of_documents = []
for item in data: 
  content = item['title'] + " \nGiá tiền: " + item['Price'] + " \nNguyên liệu: " + item['ingredients'] + "\n Công dụng: " + item['uses'] + "\n Hướng dẫn sử dụng: " + item["how to use"] + "\nHạn sử dụng (HSD): " + item["HSD"] + "\n Số lượng: " + item['quantity']
  list_of_documents.append(Document(page_content=content))

list_of_documents.append(Document(page_content=introduction))
list_of_documents.append(Document(page_content=warranty))
list_of_documents.append(Document(page_content=note))
list_of_documents.append(Document(page_content=categories))

print(len(list_of_documents))

14


In [4]:
from uuid import uuid4
from langchain_google_genai import GoogleGenerativeAIEmbeddings

uuids = [str(uuid4()) for _ in range(len(list_of_documents))]
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004", google_api_key=gemini_token)


In [6]:
# Initialize the FAISS vector store
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.faiss import DistanceStrategy

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)
vector_store.add_documents(documents=list_of_documents, ids=uuids)

['19fbe024-ca70-4736-a727-82ba80d044d0',
 '9f164dc1-74f5-4c6a-a810-7b7647171ae3',
 '58006b7d-282c-442a-9513-9a5402740a95',
 '675c73aa-839d-47a5-bf9a-2bc9a17b36e2',
 '5178727b-b945-4993-858c-0c8e1b4adb04',
 '8e54c175-dc0a-4a0c-8dc2-512171772ca0',
 'a7d005f6-d6f3-4fe2-af0b-b7619bf97c8f',
 'd76df158-ea35-4c8d-9dc7-fc38fa5ad62b',
 '14372ed2-885f-4b6f-9dc5-c0b8e976c1ee',
 '76f55817-552f-4d9e-b70e-7e963b0f08a7',
 'd2670708-ffa9-4304-bc52-04ae73fd3a21',
 '8e23328e-a6ab-4917-a8b5-13cab8ed812e',
 '8ec1171b-daf0-434b-8899-2ecfd8de48e4',
 'e5e809ce-d364-4026-8f5f-3e439687c35e']

In [7]:
# Using BM25 retriever from LangChain
from langchain_community.retrievers import BM25Retriever
bm25_retriever = BM25Retriever.from_documents(
  list_of_documents, k = 10
)

In [8]:
from pinecone import Pinecone
class Reranker:
  def __init__(self, pc_api, model = "bge-reranker-v2-m3"):
    self.pc = Pinecone(pc_api)
    self.model = model
  def compute_score(self, query, documents):
    results = self.pc.inference.rerank(
        model = self.model,
        query = query,
        documents = documents,
        parameters= {
            "truncate": "END",
        },
    )
    results_list =[]
    for r in results.data:
      results_list.append([r.score,r.document])
    return results_list
reranker = Reranker(pinecone_api)

In [9]:
class Retriever:
  def __init__(self, semantic_retriever, bm25_retriever, reranker):
    self.semantic_retriever = semantic_retriever
    self.bm25_retriever = bm25_retriever
    self.reranker = reranker

  def __call__(self,query):
    semantic_results = self.semantic_retriever.similarity_search(
      query,
      k=10,
    )
    bm25_results = self.bm25_retriever.invoke(query)

    content = set()
    retrieval_docs = []

    for result in semantic_results:
      if result.page_content not in content:
        content.add(result.page_content)
        retrieval_docs.append(result)

    for result in bm25_results:
      if result.page_content not in content:
        content.add(result.page_content)
        retrieval_docs.append(result)

    # pairs = [[query,doc.page_content] for doc in retrieval_docs]

    # scores = self.reranker.compute_score(pairs,normalize = True)
    results = reranker.compute_score(query, [doc.page_content for doc in retrieval_docs])
    scores = [r[0] for r in results]

    # Retrieve the parent document from the child chunk based on a threshold score.
    context_1 = []
    context_2 = []
    context = []
    parent_ids = set()
    for i in range(len(results)):
      # Relevance score >= 0.6 will be used as context type 1 (indicating higher relevance to the query)
      if scores[i] >= 0.6:
        context_1.append(results[i][1]['text'])
      # Relevance score >= 0.1 will be used as context type 2 (indicating medium to lower relevance to the query)
      elif scores[i] >= 0.1:
        context_2.append(results[i][1]['text'])
      # If the relevance score < 0.1, it indicates that there are no relevant documents.
    if len(context_1) > 0:
      print('Context 1')
      context=context_1
    elif len(context_2) > 0:
      print('Context 2')
      context=context_2
    else:
      print('No relevant context')
    return context

In [10]:
# Test the Retriever
retriever = Retriever(semantic_retriever = vector_store, bm25_retriever = bm25_retriever, reranker = reranker)
context = retriever("trà hoa hồng")
context

Context 1


['Các loại sản phẩm/Trà trong cửa hàng đang bán: Hoa hồng đường phèn - Hồng Tâm An, Hoa cúc táo đỏ kỳ tử - Cúc Ngọc Dưỡng Huyết,Trà an mộng ngủ ngon, Hoa hồng đường nâu - Trà dịu hồng',
 'Hoa hồng đường phèn - Hồng Tâm An \nGiá tiền: 90.000 VND \nNguyên liệu: Trà hoa hồng đường phèn được ép từ những bông hoa hồng tự nhiên được phơi hoặc sấy khô kết hợp với đường phèn và cốt trà bên trong viên trà. Nguyên liệu được trồng ở vùng núi cao Tây Bắc với khí hậu mát mẻ và nhập khẩu với nguồn gốc xuất xứ rõ ràng, chế biến theo phương pháp truyền thống kết hợp hiện đại.\n Công dụng: Những loại trà thảo mộc đường phèn ở đây đều rất tốt cho sức khỏe và mỗi loại cho ta 1 công dụng khác nhau: \n- Là thức uống tự nhiên uống thơm ngon tốt cho sức khỏe thay các loại nước ngọt hóa học \n- Tăng cường sức khỏe, tăng khả năng miễn dịch \n- Làm đẹp da, sáng mắt, thanh nhiệt, thải độc \n- Hỗ trợ ngủ ngon, tốt cho người huyết áp thấp\n Hướng dẫn sử dụng: Pha 500ml nước nóng hơn 90 độ C, đợi 7-10 phút rồi sử d

## AI agent

In [11]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool
from typing import Optional, Type, List
from langchain_core.messages import AIMessage,HumanMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# Define a BaseModel Retrieve class for LLM tool calling
class Retrieve(BaseModel):
    """
    Searches the knowledge base for answers. The query parameter should contain the contextualized question to search for in the knowledge base.
    """
    query: str = Field(description="should be a search query")

toolPrompt = PromptTemplate.from_template("""
    You are an AI assistant of a herbal tea shop.
    For any questions regarding tea products, you ***must use the `Retrieve` tool*** to obtain accurate information.
    To use the `Retrieve` tool, take the user's most recent question as well as relevant chat history, extract a clear, concise search query from the question and chat context. Pass this query to the `Retrieve` tool by setting the `query` parameter.
    For all other questions or general interactions, such as greetings, expressions of gratitude, or requests for life advice or general knowledge, you may reply directly.
    \\n Your customers are Vietnamese, so always reply in Vietnamese.
    \\n Here is your chat history: {chat_history}
""")

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=gemini_token, temperature = 0.5)
agent = llm.bind_tools([Retrieve])
agentChain = toolPrompt | agent

c:\Users\huyho\anaconda3\envs\text2sql\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


## Expose API

In [12]:
import os
import json
from pyngrok import ngrok
from flask import Flask, jsonify, request
from flask_cors import CORS

In [13]:
answerModel = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=gemini_token, temperature = 0.5)
answerPrompt = PromptTemplate.from_template("""
     Bạn là chuyên viên tư vấn khách hàng của tiệm trà thảo mộc HerbaZen.
     Câu hỏi của khách hàng: {query}\nTrả lời câu hỏi dựa vào các thông tin sản phẩm của cửa hàng dưới đây: {source_information}.\
 """)
answerChain = answerPrompt | answerModel

In [14]:
ngrok.set_auth_token(ngrok_token)

app = Flask(__name__)
CORS(app)

@app.route('/v1/chat', methods=['POST'])
def chat_v1():
    # Extract parameters from the request
    user_message = request.json.get('message', {})
    chat_history = request.json.get('context', [])
    stream = True  # Default to False if not provided

    print(f'Message: {user_message}')
    print(f'Chat History: {chat_history}')

    history_string = "\n".join(
      [
          f"{message['role']} : {message['content']}"
          for message in chat_history
      ]
    )
    agent_response = agentChain.invoke({"chat_history":history_string})

    print(agent_response)
    if 'function_call' in agent_response.additional_kwargs:
      print('Guide to Herbal Tea')
      refine_query = agent_response.tool_calls[0]['args']['query']
      print(f'REFINED: {refine_query}')
      context = retriever(refine_query)
      source_information = ""
      for doc in context:
        content = doc
        source_information += content + "\n"

      if stream:
        def generate():
          for chunk in answerChain.stream({"query": refine_query, "source_information": source_information, "chat_history":history_string}):
            yield chunk.content
        return app.response_class(generate(), mimetype='text/plain')
      else:
        reponse = answerChain.invoke({"query": refine_query, "source_information": source_information,"chat_history":history_string})
        return jsonify({'response': reponse.content})
    else:
      print('Guide to CHAT')
      if stream:
        def generate():
          for chunk in agent_response.content.split(" "):
            yield chunk + " "
        return app.response_class(generate(), mimetype='text/plain')
      else:
        return jsonify({'response': agent_response.content})
    
if __name__ == '__main__':
    # Start ngrok to tunnel the Flask app
    url = ngrok.connect(5000)
    print(f" * ngrok tunnel: {url}")

    # Start Flask app
    app.run(port=5000)

 * ngrok tunnel: NgrokTunnel: "https://b56a-42-113-131-6.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Jun/2025 15:43:46] "OPTIONS /v1/chat HTTP/1.1" 200 -


Message: xin chào
Chat History: [{'role': 'user', 'content': 'xin chào'}]


127.0.0.1 - - [27/Jun/2025 15:43:48] "POST /v1/chat HTTP/1.1" 200 -


content='Chào bạn!' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--1aee1b09-4235-4721-bae7-bcbece2c107d-0' usage_metadata={'input_tokens': 190, 'output_tokens': 5, 'total_tokens': 195, 'input_token_details': {'cache_read': 0}}
Guide to CHAT


...

content='Chào bạn!' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--085c3323-a80a-43c9-9cd5-adb1172f2015-0' usage_metadata={'input_tokens': 190, 'output_tokens': 5, 'total_tokens': 195, 'input_token_details': {'cache_read': 0}}
Guide to CHAT


127.0.0.1 - - [27/Jun/2025 16:33:22] "OPTIONS /v1/chat HTTP/1.1" 200 -


Message: xin chào
Chat History: [{'role': 'user', 'content': 'xin chào'}]


127.0.0.1 - - [27/Jun/2025 16:33:23] "POST /v1/chat HTTP/1.1" 200 -


content='Chào bạn!' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--2062585b-de2f-4566-a5ba-cf2259ce1dcd-0' usage_metadata={'input_tokens': 190, 'output_tokens': 5, 'total_tokens': 195, 'input_token_details': {'cache_read': 0}}
Guide to CHAT
